## Making a Request and getting a Response

In [2]:
import requests as req
data = req.get('https://api.github.com/events') # data is a Response object
data

<Response [200]>

For POST request:

In [11]:
post_response = req.post('http://httpbin.org/post', data = {'key':'value'})
post_response

<Response [200]>

We can also attach request parameters to the GET request:

In [7]:
param_get_response = req.get('http://httpbin.org/get', params = {'key':'value'})
param_get_response

<Response [200]>

In [26]:
image_response = req.get('https://www.gstatic.com/webp/gallery3/1.png')
image_response

<Response [200]>

All production code should supply timeout time:

In [4]:
try:
    github_response = req.get('http://github.com', timeout=0.001)
except:
    print("Tired of waiting for response")

Tired of waiting for response


## The Response Object

To get the URL of the corresponding request:

In [15]:
param_get_response.url

'http://httpbin.org/get?key=value'

To get the response content, first look at the Content Type of the response.

In [21]:
for key,value in param_get_response.headers.items():
    print(key +' : '+ value, end = ',\t')

Connection : keep-alive,	Server : meinheld/0.6.1,	Date : Tue, 23 Jan 2018 20:01:02 GMT,	Content-Type : application/json,	Access-Control-Allow-Origin : *,	Access-Control-Allow-Credentials : true,	X-Powered-By : Flask,	X-Processed-Time : 0.00111198425293,	Content-Length : 298,	Via : 1.1 vegur,	

Or

In [8]:
param_get_response.headers['Content-Type']

'application/json'

Also check the encoding

In [24]:
print(param_get_response.encoding)

None


Print response text

In [25]:
param_get_response.text

'{\n  "args": {\n    "key": "value"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Connection": "close", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.11.1"\n  }, \n  "origin": "183.83.66.234", \n  "url": "http://httpbin.org/get?key=value"\n}\n'

To get the associated binary data:

In [31]:
param_get_response.content

b'{\n  "args": {\n    "key": "value"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Connection": "close", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.11.1"\n  }, \n  "origin": "183.83.66.234", \n  "url": "http://httpbin.org/get?key=value"\n}\n'

In the above case the response is json, so to get json object, use *json()* function

In [32]:
param_get_response.json()

{'args': {'key': 'value'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'close',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.11.1'},
 'origin': '183.83.66.234',
 'url': 'http://httpbin.org/get?key=value'}

In case of an image response, the content-type is image/png:

In [27]:
for key,value in image_response.headers.items():
    print(key +' : '+ value, end = ',\t')

Accept-Ranges : bytes,	Content-Type : image/png,	Vary : Origin,	Content-Length : 121363,	Date : Tue, 23 Jan 2018 09:51:12 GMT,	Expires : Wed, 23 Jan 2019 09:51:12 GMT,	Last-Modified : Thu, 21 Apr 2016 03:17:22 GMT,	X-Content-Type-Options : nosniff,	Server : sffe,	X-XSS-Protection : 1; mode=block,	Cache-Control : public, max-age=31536000,	Age : 37411,	Alt-Svc : hq=":443"; ma=2592000; quic=51303431; quic=51303339; quic=51303338; quic=51303337; quic=51303335,quic=":443"; ma=2592000; v="41,39,38,37,35",	

In [ ]:
image_response.text # will output some garbage text, since it is a binary

In [ ]:
image_response.content # binary data associated with the image

In [30]:
with open('image.png', 'wb') as f:   # save the binary object (image in this case) as a file
    f.write(image_response.content)

## Custom Headers

To add custom request headers use the headers keyword argument.

In [34]:
headers = {'browser':'Mozilla'}
req.get('https://www.google.com', headers=headers)

<Response [200]>

To download a range of bytes add Range keyword to the request:

In [ ]:
resp = req.get('https://url.com', headers = {'Range':'bytes=0-512'})

If the response is 206 then the server supports partial download, else if the response is 200 then the entire file will be downloaded.

## POST Request

To send an image as part of request:

In [6]:
image_file = open('image.png', 'rb')
files = {'file': image_file}
image_send_response = req.post('http://httpbin.org/post', files=files)

## Cookies

If the response contains cookies:

In [9]:
param_get_response.cookies

<RequestsCookieJar[]>

In [12]:
search_response = req.get('https://www.google.co.in/search?q=cookies')
for name,content in search_response.cookies.items():
    print('Name: ' + name + '\t Content: ' + content)

Name: 1P_JAR	 Content: 2018-01-23-20
Name: NID	 Content: 122=Sgt25sQEjQUhf_S-1xkCKVuhQGMjjBEGwPj9SiClX8wnYI_KnPzy5sCm_d1nMwN_oMAOHV0vSH5Cpdrc3-XgNBJZufpiaabz85a0RQL13d9swCsgE02jEes2NixUQMSJ


## Streaming Downloads

With *stream=True* option, only the response header is downloaded not the response content.

In [26]:
file_response = req.get('https://download-installer.cdn.mozilla.net/pub/firefox/releases/58.0/win32/en-US/Firefox%20Installer.exe',
                        stream=True)
with open('firefox_installer.exe', 'wb') as f:
    for chunk in file_response.iter_content(chunk_size=512):
        if chunk:
            f.write(chunk)

## Effort On Multithreaded Download

In [29]:
import threading
import requests as req
import time
import os

start_time = time.time()
def download(url, chunk_sz, start, end, i):
    print('Start byte={0}, End byte={1}'.format(start, end))
    response = req.get(url, stream=True, headers={'Range':'bytes='+str(start)+'-'+str(end)})
    with open('file_'+str(i)+'.exe', 'wb') as f:
        for chunk in response.iter_content(chunk_size=chunk_sz):
            if chunk:
                f.write(chunk)

url = 'http://www.voobly.com/updates/voobly-v2.2.5.57.exe'                
size = int(req.get(url, stream=True).headers['Content-length'])
print('Total size = '+str(size))
chunk_size = int(size/5)

for i in range(5):
    if i==0:
        threading.Thread(target=download(url, chunk_size, i*chunk_size, (i+1)*chunk_size, i)).start()
    elif i<4:
        threading.Thread(target=download(url, chunk_size, 1+(i*chunk_size), (i+1)*chunk_size, i)).start()
    else:
        threading.Thread(target=download(url, chunk_size, i*chunk_size, size, i)).start()
    print('Total threads = ' + str(threading.active_count()))

end_time = time.time()
print(end_time-start_time)
        
def combine(file1, file2):
    with open(file1, 'ab') as primary, open(file2, 'rb') as secondary:
        primary.write(secondary.read())
    os.remove(file2)
        
for i in range(1,5):
    combine('file_0.exe', 'file_'+str(i)+'.exe')


Total size = 10474190
Start byte=0, End byte=2094838
Total threads = 5
Start byte=2094839, End byte=4189676
Total threads = 5
Start byte=4189677, End byte=6284514
Total threads = 5
Start byte=6284515, End byte=8379352
Total threads = 5
Start byte=8379352, End byte=10474190
Total threads = 5
11.292553663253784


In [27]:
start_time = time.time()

response = req.get(url)
with open('file_1.exe', 'wb') as f:
    f.write(response.content)
    
end_time = time.time()
print(end_time-start_time)

4.549771308898926


In [26]:
import multiprocessing
import requests as req
import time
import os

start_time = time.time()
def download(url, chunk_sz, start, end, i):
    print('Start byte={0}, End byte={1}'.format(start, end))
    response = req.get(url, stream=True, headers={'Range':'bytes='+str(start)+'-'+str(end)})
    with open('file_'+str(i)+'.exe', 'wb') as f:
        for chunk in response.iter_content(chunk_size=chunk_sz):
            if chunk:
                f.write(chunk)

url = 'http://www.voobly.com/updates/voobly-v2.2.5.57.exe'                
size = int(req.get(url, stream=True).headers['Content-length'])
print('Total size = '+str(size))
chunk_size = int(size/5)

for i in range(5):
    if i==0:
        multiprocessing.Process(target=download(url, chunk_size, i*chunk_size, (i+1)*chunk_size, i)).start()
    elif i<4:
        multiprocessing.Process(target=download(url, chunk_size, 1+(i*chunk_size), (i+1)*chunk_size, i)).start()
    else:
        multiprocessing.Process(target=download(url, chunk_size, i*chunk_size, size, i)).start()

end_time = time.time()
print(end_time-start_time)
        
def combine(file1, file2):
    with open(file1, 'ab') as primary, open(file2, 'rb') as secondary:
        primary.write(secondary.read())
    os.remove(file2)
        
for i in range(1,5):
    combine('file_0.exe', 'file_'+str(i)+'.exe')

Total size = 10474190
Start byte=0, End byte=2094838
Start byte=2094839, End byte=4189676
Start byte=4189677, End byte=6284514
Start byte=6284515, End byte=8379352
Start byte=8379352, End byte=10474190
12.03153657913208
